In [1]:
import os
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import neighbors
import seaborn as sns
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import os
os.getcwd()
os.chdir('/home/ghk829/zillow')
print('Loading Properties ...')
properties2016 = pd.read_csv('./properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('./properties_2017.csv', low_memory = False)

print('Loading Train ...')
train2016 = pd.read_csv('./train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('./train_2017.csv', parse_dates=['transactiondate'], low_memory=False)
test_df = pd.read_csv('./sample_submission.csv', low_memory=False)
properties = pd.read_csv('./properties_2016.csv', low_memory=False)
# field is named differently in submission
test_df['parcelid'] = test_df['ParcelId']


Loading Properties ...
Loading Train ...


In [2]:
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = (df["transactiondate"].dt.year - 2016)*4 +df["transactiondate"].dt.quarter
    return df

In [3]:
def fillna_knn( df, base, target, fraction = 1, threshold = 10, n_neighbors = 5 ):
    assert isinstance( base , list ) or isinstance( base , np.ndarray ) and isinstance( target, str ) 
    whole = [ target ] + base
    
    miss = df[target].isnull()
    notmiss = ~miss 
    nummiss = miss.sum()
    
    enc = OneHotEncoder()
    X_target = df.loc[ notmiss, whole ].sample( frac = fraction )
    
    enc.fit( X_target[ target ].unique().reshape( (-1,1) ) )
    
    Y = enc.transform( X_target[ target ].values.reshape((-1,1)) ).toarray()
    X = X_target[ base  ]
    
    print( 'fitting' )
    n_neighbors = n_neighbors
    clf = neighbors.KNeighborsClassifier( n_neighbors, weights = 'uniform' )
    clf.fit( X, Y )
    
    print( 'the shape of active features: ' ,enc.active_features_.shape )
    
    print( 'predicting' )
    Z = clf.predict(df.loc[miss, base])
    
    numunperdicted = Z[:,0].sum()
    if numunperdicted / nummiss *100 < threshold :
        print( 'writing result to df' )    
        df.loc[ miss, target ]  = np.dot( Z , enc.active_features_ )
        print( 'num of unperdictable data: ', numunperdicted )
        return enc
    else:
        print( 'out of threshold: {}% > {}%'.format( numunperdicted / nummiss *100 , threshold ) )

#function to deal with variables that are actually string/categories
def zoningcode2int( df, target ):
    storenull = df[ target ].isnull()
    enc = LabelEncoder( )
    df[ target ] = df[ target ].astype( str )

    print('fit and transform')
    df[ target ]= enc.fit_transform( df[ target ].values )
    print( 'num of categories: ', enc.classes_.shape  )
    df.loc[ storenull, target ] = np.nan
    print('recover the nan value')
    return enc


In [4]:
train2016 = add_date_features(train2016)
train2017 = add_date_features(train2017)

In [5]:
import numpy as np
print('Merge Train with Properties ...')
train2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')
train2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

print('Tax Features 2017  ...')
train2017.iloc[:, train2017.columns.str.startswith('tax')] = np.nan

print('Concat Train 2016 & 2017 ...')
train_df = pd.concat([train2016, train2017], axis = 0)
test_df = pd.merge(test_df[['ParcelId']], properties2016.rename(columns = {'parcelid': 'ParcelId'}), how = 'left', on = 'ParcelId')


Merge Train with Properties ...
Tax Features 2017  ...
Concat Train 2016 & 2017 ...


In [6]:
import gc
gc.collect()

68

In [17]:
train_df=train_df[train_df.latitude.notnull()]

In [18]:
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'buildingqualitytypeid', fraction = 0.15, n_neighbors = 1 )


zoningcode2int( df = train_df,
                            target = 'propertycountylandusecode' )
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'propertycountylandusecode', fraction = 0.15, n_neighbors = 1 )

zoningcode2int( df = train_df,
                            target = 'propertyzoningdesc' )

fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'propertyzoningdesc', fraction = 0.15, n_neighbors = 1 )

#regionidcity, regionidneighborhood & regionidzip - assume it is the same as the nereast property. 
#As mentioned above, this is ok if there's a property very nearby to the one with missing values (I leave it up to the reader to check if this is the case!)
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidcity', fraction = 0.15, n_neighbors = 1 )

fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidneighborhood', fraction = 0.15, n_neighbors = 1 )

fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidzip', fraction = 0.15, n_neighbors = 1 )

#unitcnt - the number of structures the unit is built into. Assume it is the same as the nearest properties. If the property with missing values is in a block of flats or in a terrace street then this is probably ok - but again I leave it up to the reader to check if this is the case!
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'unitcnt', fraction = 0.15, n_neighbors = 1 )

#yearbuilt - assume it is the same as the nearest property. This assumes properties all near to each other were built around the same time
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'yearbuilt', fraction = 0.15, n_neighbors = 1 )

#lot size square feet - not sure what to do about this one. Lets use nearest neighbours. Assume it has same lot size as property closest to it
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'lotsizesquarefeet', fraction = 0.15, n_neighbors = 1 )


fitting
('the shape of active features: ', (12,))
predicting
writing result to df


/home/ghk829/miniconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/home/ghk829/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('num of unperdictable data: ', 158.0)
fit and transform

/home/ghk829/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



('num of categories: ', (91,))
recover the nan value
fitting
('the shape of active features: ', (58,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fit and transform
('num of categories: ', (2347,))
recover the nan value
fitting
('the shape of active features: ', (1321,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fitting
('the shape of active features: ', (171,))
predicting
writing result to df
('num of unperdictable data: ', 2.0)
fitting
('the shape of active features: ', (426,))
predicting
writing result to df
('num of unperdictable data: ', 502.0)
fitting
('the shape of active features: ', (386,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fitting
('the shape of active features: ', (4,))
predicting
writing result to df
('num of unperdictable data: ', 5559.0)
fitting
('the shape of active features: ', (129,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fitting
('the shape of active 

OneHotEncoder(categorical_features='all', dtype=<type 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [21]:
print ("Remove features with one unique value !!")
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print("We exclude: %s" % len(exclude_unique))


Remove features with one unique value !!
We exclude: 9


In [22]:
print ("Define training features !!")
exclude_other = ['parcelid', 'logerror','propertyzoningdesc']
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other and c not in exclude_unique:
        train_features.append(c)
print("We use these for training: %s" % len(train_features))

Define training features !!
We use these for training: 43


In [23]:
print ("Replacing NaN values by 0 !!")
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)


print ("remove outliers")
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]

train_df=train_df.assign(diff_cal_fin=lambda x: x.calculatedfinishedsquarefeet-x.finishedfloor1squarefeet)
new_cols=['finishedfloor1squarefeet',
'garagetotalsqft',
'lotsizesquarefeet',
'poolsizesum']
for i,value in enumerate(new_cols):
    print(str(i)+":"+value)
    train_df=eval("train_df.assign(new_{}=lambda x: x.{} /x.calculatedfinishedsquarefeet)".format(value,value))

VAL_SPLIT_DATE = '2016-09-15'   # Cutoff date for validation split
select_qtr4 = train_df["transactiondate"] >= VAL_SPLIT_DATE
valid = train_df[select_qtr4]
train = train_df[~select_qtr4]
valid.drop(["transactiondate"], inplace=True, axis=1)
train.drop(["transactiondate"], inplace=True, axis=1)
print("Train: ", train.shape)
print("Test: ", valid.shape)

test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)

Replacing NaN values by 0 !!


/home/ghk829/miniconda2/lib/python2.7/site-packages/pandas/core/frame.py:2754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


remove outliers
0:finishedfloor1squarefeet
1:garagetotalsqft
2:lotsizesquarefeet
3:poolsizesum


/home/ghk829/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ghk829/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Train: ', (74478, 68))
('Test: ', (89971, 68))


In [24]:
train_features.remove('transactiondate')
print ("Define categorial features !!")
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])


Define categorial features !!
Cat features are: ['transaction_year', 'transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt', 'assessmentyear']


In [25]:
train_features_new = train_features+['new_finishedfloor1squarefeet',
'new_garagetotalsqft',
'new_lotsizesquarefeet',
'new_poolsizesum','diff_cal_fin']

In [26]:
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error

In [27]:
X_train = train[train_features_new]
y_train = train.logerror
X_valid = valid[train_features_new]
y_valid = valid.logerror
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

((74478, 47), (74478,))
((89971, 47), (89971,))


In [28]:
num_ensembles = 3
tree_counts = []
MAEs = []
for i in range(num_ensembles):
    # TODO(you): Use CV, tune hyperparameters
    model = CatBoostRegressor(
        iterations=200, learning_rate=0.004,
        depth=6, l2_leaf_reg=15,
        bagging_temperature=8,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model.fit(
        X_train, y_train,
        eval_set=[X_valid, y_valid],
        cat_features=cat_feature_inds,
#        verbose=True,
        use_best_model=True
        )
    tree_counts.append( model.tree_count_ )
    MAEs.append( mean_absolute_error(y_valid, model.predict(X_valid)) )
print(MAEs)

[0.052331489140260055, 0.052346852539937538, 0.052348389715953444]


In [29]:
test_df=test_df.assign(diff_cal_fin=lambda x: x.calculatedfinishedsquarefeet-x.finishedfloor1squarefeet)
new_cols=['finishedfloor1squarefeet',
'garagetotalsqft',
'lotsizesquarefeet',
'poolsizesum']
for i,value in enumerate(new_cols):
    print(str(i)+":"+value)
    test_df=eval("test_df.assign(new_{}=lambda x: x.{} /x.calculatedfinishedsquarefeet)".format(value,value))

0:finishedfloor1squarefeet
1:garagetotalsqft
2:lotsizesquarefeet
3:poolsizesum


In [30]:
X_test=test_df[train_features_new]
y_pred=model.predict(X_test)

In [31]:
submission = pd.DataFrame({
    'ParcelId': test_df['ParcelId'],
})
test_dates = {
    '201610': pd.Timestamp('2016-10-01'),
    '201611': pd.Timestamp('2016-11-01'),
    '201612': pd.Timestamp('2016-12-01'),
    '201710': pd.Timestamp('2017-10-01'),
    '201711': pd.Timestamp('2017-11-01'),
    '201712': pd.Timestamp('2017-12-02')
}
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    submission[label] = y_pred

Predicting for: 201612 ... 
Predicting for: 201610 ... 
Predicting for: 201611 ... 
Predicting for: 201712 ... 
Predicting for: 201711 ... 
Predicting for: 201710 ... 


In [32]:
train_df=train2016
print('Remove missing data fields ...')

missing_perc_thresh = 0.98
exclude_missing = []
num_rows = train_df.shape[0]
for c in train_df.columns:
    num_missing = train_df[c].isnull().sum()
    if num_missing == 0:
        continue
    missing_frac = num_missing / float(num_rows)
    if missing_frac > missing_perc_thresh:
        exclude_missing.append(c)
print("We exclude: %s" % len(exclude_missing))

del num_rows, missing_perc_thresh
gc.collect();
train_df=train_df[train_df.latitude.notnull()]
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'buildingqualitytypeid', fraction = 0.15, n_neighbors = 1 )


zoningcode2int( df = train_df,
                            target = 'propertycountylandusecode' )
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'propertycountylandusecode', fraction = 0.15, n_neighbors = 1 )

zoningcode2int( df = train_df,
                            target = 'propertyzoningdesc' )

fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'propertyzoningdesc', fraction = 0.15, n_neighbors = 1 )

#regionidcity, regionidneighborhood & regionidzip - assume it is the same as the nereast property. 
#As mentioned above, this is ok if there's a property very nearby to the one with missing values (I leave it up to the reader to check if this is the case!)
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidcity', fraction = 0.15, n_neighbors = 1 )

fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidneighborhood', fraction = 0.15, n_neighbors = 1 )

fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'regionidzip', fraction = 0.15, n_neighbors = 1 )

#unitcnt - the number of structures the unit is built into. Assume it is the same as the nearest properties. If the property with missing values is in a block of flats or in a terrace street then this is probably ok - but again I leave it up to the reader to check if this is the case!
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'unitcnt', fraction = 0.15, n_neighbors = 1 )

#yearbuilt - assume it is the same as the nearest property. This assumes properties all near to each other were built around the same time
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'yearbuilt', fraction = 0.15, n_neighbors = 1 )

#lot size square feet - not sure what to do about this one. Lets use nearest neighbours. Assume it has same lot size as property closest to it
fillna_knn( df = train_df,
                  base = [ 'latitude', 'longitude' ] ,
                  target = 'lotsizesquarefeet', fraction = 0.15, n_neighbors = 1 )

print ("Remove features with one unique value !!")
exclude_unique = []
for c in train_df.columns:
    num_uniques = len(train_df[c].unique())
    if train_df[c].isnull().sum() != 0:
        num_uniques -= 1
    if num_uniques == 1:
        exclude_unique.append(c)
print("We exclude: %s" % len(exclude_unique))
print ("Define training features !!")
exclude_other = ['parcelid', 'logerror','propertyzoningdesc']
train_features = []
for c in train_df.columns:
    if c not in exclude_missing \
       and c not in exclude_other and c not in exclude_unique:
        train_features.append(c)
print("We use these for training: %s" % len(train_features))
print ("Replacing NaN values by 0 !!")
train_df.fillna(0, inplace=True)
test_df.fillna(0, inplace=True)


print ("remove outliers")
train_df=train_df[ train_df.logerror > -0.4 ]
train_df=train_df[ train_df.logerror < 0.419 ]

train_df=train_df.assign(diff_cal_fin=lambda x: x.calculatedfinishedsquarefeet-x.finishedfloor1squarefeet)
new_cols=['finishedfloor1squarefeet',
'garagetotalsqft',
'lotsizesquarefeet',
'poolsizesum']
for i,value in enumerate(new_cols):
    print(str(i)+":"+value)
    train_df=eval("train_df.assign(new_{}=lambda x: x.{} /x.calculatedfinishedsquarefeet)".format(value,value))

VAL_SPLIT_DATE = '2016-09-15'   # Cutoff date for validation split
select_qtr4 = train_df["transactiondate"] >= VAL_SPLIT_DATE
valid = train_df[select_qtr4]
train = train_df[~select_qtr4]
valid.drop(["transactiondate"], inplace=True, axis=1)
train.drop(["transactiondate"], inplace=True, axis=1)
print("Train: ", train.shape)
print("Test: ", valid.shape)

test_df['transactiondate'] = pd.Timestamp('2016-12-01') 
test_df = add_date_features(test_df)
train_features.remove('transactiondate')
print ("Define categorial features !!")
cat_feature_inds = []
cat_unique_thresh = 1000
for i, c in enumerate(train_features):
    num_uniques = len(train_df[c].unique())
    if num_uniques < cat_unique_thresh \
       and not 'sqft' in c \
       and not 'cnt' in c \
       and not 'nbr' in c \
       and not 'number' in c:
        cat_feature_inds.append(i)
        
print("Cat features are: %s" % [train_features[ind] for ind in cat_feature_inds])
train_features_new = train_features+['new_finishedfloor1squarefeet',
'new_garagetotalsqft',
'new_lotsizesquarefeet',
'new_poolsizesum','diff_cal_fin']
print train_features_new
X_train = train[train_features_new]
y_train = train.logerror
X_valid = valid[train_features_new]
y_valid = valid.logerror
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

Remove missing data fields ...
We exclude: 15
fitting
('the shape of active features: ', (6,))
predicting
writing result to df
('num of unperdictable data: ', 221.0)
fit and transform
('num of categories: ', (78,))
recover the nan value
fitting
('the shape of active features: ', (46,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fit and transform
('num of categories: ', (1997,))
recover the nan value
fitting
('the shape of active features: ', (1075,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fitting
('the shape of active features: ', (170,))
predicting
writing result to df
('num of unperdictable data: ', 2.0)
fitting
('the shape of active features: ', (380,))
predicting
writing result to df
('num of unperdictable data: ', 222.0)
fitting
('the shape of active features: ', (378,))
predicting
writing result to df
('num of unperdictable data: ', 0.0)
fitting
('the shape of active features: ', (4,))
predicting
out of threshold: 18.4512248

/home/ghk829/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ghk829/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


('Train: ', (74478, 68))
('Test: ', (14050, 68))
Define categorial features !!
Cat features are: ['transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'buildingqualitytypeid', 'fips', 'heatingorsystemtypeid', 'propertycountylandusecode', 'propertylandusetypeid', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'yearbuilt']
['transaction_month', 'transaction_day', 'transaction_quarter', 'airconditioningtypeid', 'bathroomcnt', 'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr', 'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'finishedsquarefeet15', 'finishedsquarefeet50', 'fips', 'fireplacecnt', 'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet', 'propertycountylandusecode', 'propertylandusetypeid', 'rawcensustractandblock', 'regionidcity', 'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt', 'threequart

In [33]:
num_ensembles = 3
tree_counts = []
MAEs = []
for i in range(num_ensembles):
    # TODO(you): Use CV, tune hyperparameters
    model2 = CatBoostRegressor(
        iterations=200, learning_rate=0.004,
        depth=6, l2_leaf_reg=15,
        bagging_temperature=8,
        loss_function='MAE',
        eval_metric='MAE',
        random_seed=i)
    model2.fit(
        X_train, y_train,
        eval_set=[X_valid, y_valid],
        cat_features=cat_feature_inds,
#        verbose=True,
        use_best_model=True
        )
    tree_counts.append( model2.tree_count_ )
    MAEs.append( mean_absolute_error(y_valid, model2.predict(X_valid)) )
print(MAEs)

[0.051815088623889873, 0.051819626203590659, 0.051794808012161873]


In [34]:
test_df=test_df.assign(diff_cal_fin=lambda x: x.calculatedfinishedsquarefeet-x.finishedfloor1squarefeet)
new_cols=['finishedfloor1squarefeet',
'garagetotalsqft',
'lotsizesquarefeet',
'poolsizesum']
for i,value in enumerate(new_cols):
    print(str(i)+":"+value)
    test_df=eval("test_df.assign(new_{}=lambda x: x.{} /x.calculatedfinishedsquarefeet)".format(value,value))

0:finishedfloor1squarefeet
1:garagetotalsqft
2:lotsizesquarefeet
3:poolsizesum


In [35]:
X_test=test_df[train_features_new]
y_pred=model2.predict(X_test)

In [36]:
test_dates = {
    '201610': pd.Timestamp('2016-10-01'),
    '201611': pd.Timestamp('2016-11-01'),
    '201612': pd.Timestamp('2016-12-01')
}
for label, test_date in test_dates.items():
    print("Predicting for: %s ... " % (label))
    submission[label] = y_pred

Predicting for: 201612 ... 
Predicting for: 201610 ... 
Predicting for: 201611 ... 


In [37]:
submission.to_csv('./final_solution_0.csv', float_format='%.6f',index=False)

In [22]:
os.getcwd()

'/home/ghk829/zillow'

In [25]:
train_df.columns

Index([u'parcelid', u'logerror', u'transactiondate', u'transaction_year',
       u'transaction_month', u'transaction_day', u'transaction_quarter',
       u'airconditioningtypeid', u'architecturalstyletypeid', u'basementsqft',
       u'bathroomcnt', u'bedroomcnt', u'buildingclasstypeid',
       u'buildingqualitytypeid', u'calculatedbathnbr', u'decktypeid',
       u'finishedfloor1squarefeet', u'calculatedfinishedsquarefeet',
       u'finishedsquarefeet12', u'finishedsquarefeet13',
       u'finishedsquarefeet15', u'finishedsquarefeet50',
       u'finishedsquarefeet6', u'fips', u'fireplacecnt', u'fullbathcnt',
       u'garagecarcnt', u'garagetotalsqft', u'hashottuborspa',
       u'heatingorsystemtypeid', u'latitude', u'longitude',
       u'lotsizesquarefeet', u'poolcnt', u'poolsizesum', u'pooltypeid10',
       u'pooltypeid2', u'pooltypeid7', u'propertycountylandusecode',
       u'propertylandusetypeid', u'propertyzoningdesc',
       u'rawcensustractandblock', u'regionidcity', u'regionidcou